<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/sweet_potato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

고구마 가격 예측 문제

In [1]:
!pip uninstall --y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=a8f345043383215e959f84f3183b2b18c13db23c0468499968fd094c8b8248b3
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 29 12:48 kaggle.json


In [35]:
!kaggle competitions download -c 2020termproject-18011826

  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 9.55MB/s]


In [36]:
!unzip 2020termproject-18011826.zip

Archive:  2020termproject-18011826.zip
  inflating: submit_sample.csv       
  inflating: test_sweetpotato_price.csv  
  inflating: train_sweetpotato_price.csv  


In [37]:
import pandas as pd 
import numpy as np
import torch 
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

In [38]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [39]:
train = pd.read_csv('train_sweetpotato_price.csv')
print(train.head(10))
print(train.info())


       year  avgTemp  minTemp  maxTemp  rainFall  avgPrice
0  20100101     -7.6    -12.7     -3.6       0.0      3520
1  20100102     -3.6     -7.4      0.2       1.4      3520
2  20100103     -6.8    -10.5     -3.2       0.0      3520
3  20100104     -5.9     -8.0     -3.4      14.2      3520
4  20100105     -9.9    -12.3     -7.0       0.0      3520
5  20100106    -11.2    -13.3     -8.1       0.0      3520
6  20100107    -10.1    -13.6     -5.5       0.0      4038
7  20100108     -8.0    -11.8     -3.3       0.0      4038
8  20100109     -5.1     -8.7     -1.9       0.4      4038
9  20100110     -3.4     -5.3     -0.3       0.0      4038
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      1096 non-null   int64  
 1   avgTemp   1096 non-null   float64
 2   minTemp   1096 non-null   float64
 3   maxTemp   1096 non-null   float64
 4   rainFa

In [89]:
learning_rate = 0.001
training_epoch = 2000
batch_size = 50 

In [90]:
x_train = train.iloc[:,1:-1]
y_train = train.iloc[:,[-1]]

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

print(x_train.shape)
print(y_train.shape)

torch.Size([1096, 4])
torch.Size([1096, 1])


In [91]:
train_dataset = torch.utils.data.TensorDataset(x_train,y_train)

data_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last=True)



layer 5개 , kaiming uniform, kaiming normal 로 초기화

In [92]:
linear1 = nn.Linear(4,8,bias=True)
linear2 = nn.Linear(8,8,bias=True)
linear3 = nn.Linear(8,4,bias=True)
linear4 = nn.Linear(4,4,bias=True)
linear5 = nn.Linear(4,1,bias=True)

nn.init.kaiming_uniform_(linear1.weight)
nn.init.kaiming_normal_(linear2.weight)
nn.init.kaiming_uniform_(linear3.weight)
nn.init.kaiming_normal_(linear4.weight)
nn.init.kaiming_uniform_(linear5.weight)
selu = nn.SELU()

activation function - selu

In [93]:
model = nn.Sequential(
    linear1,selu,
    linear2,selu,
    linear3,selu,
    linear4,selu,
    linear5
).to(device)

In [94]:
loss = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [95]:
total_batch = len(data_loader)

for epoch in range(training_epoch):
    avg_cost = 0
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis,Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('epoch {:.4f} , cost = {:.6f}'.format(epoch,avg_cost))
print('learning finished!')

epoch 0.0000 , cost = 28278900.000000
epoch 1.0000 , cost = 28094362.000000
epoch 2.0000 , cost = 28135248.000000
epoch 3.0000 , cost = 28349262.000000
epoch 4.0000 , cost = 28286228.000000
epoch 5.0000 , cost = 27973806.000000
epoch 6.0000 , cost = 27974246.000000
epoch 7.0000 , cost = 27955354.000000
epoch 8.0000 , cost = 28062534.000000
epoch 9.0000 , cost = 27891604.000000
epoch 10.0000 , cost = 27326242.000000
epoch 11.0000 , cost = 26741434.000000
epoch 12.0000 , cost = 25105616.000000
epoch 13.0000 , cost = 23434282.000000
epoch 14.0000 , cost = 21369470.000000
epoch 15.0000 , cost = 18636184.000000
epoch 16.0000 , cost = 15474760.000000
epoch 17.0000 , cost = 12171192.000000
epoch 18.0000 , cost = 9303772.000000
epoch 19.0000 , cost = 7025650.500000
epoch 20.0000 , cost = 5470886.500000
epoch 21.0000 , cost = 4474786.500000
epoch 22.0000 , cost = 3992778.500000
epoch 23.0000 , cost = 3728878.000000
epoch 24.0000 , cost = 3582044.250000
epoch 25.0000 , cost = 3514175.750000
epoc

In [96]:
test = pd.read_csv('test_sweetpotato_price.csv')
print(test.info())
test = test.iloc[:,1:]
test = np.array(test)
test = torch.FloatTensor(test).to(device)

with torch.no_grad():
    predict = model(test)
predict

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      6 non-null      int64  
 1   avgTemp   6 non-null      float64
 2   minTemp   6 non-null      float64
 3   maxTemp   6 non-null      float64
 4   rainFall  6 non-null      float64
dtypes: float64(4), int64(1)
memory usage: 368.0 bytes
None


tensor([[4534.3516],
        [4410.3521],
        [4704.2324],
        [4927.7593],
        [5298.0454],
        [6025.5488]], device='cuda:0')

In [97]:
correct_prediction = predict.cpu().numpy().reshape(-1,1)
result = pd.read_csv('submit_sample.csv')

In [98]:
for i in range(len(correct_prediction)):
    result['Expected'][i] = correct_prediction[i]
    

In [99]:
result.to_csv('submit.csv',index=False)

In [100]:
!kaggle competitions submit -c 2020termproject-18011826 -f submit.csv -m "14010974_이기택_기말고사파트2-2"

100% 54.0/54.0 [00:05<00:00, 9.58B/s]
Successfully submitted to SejongAI.텀프로젝트.[고구마 가격 예측 문제]